In [7]:
from datasets import load_dataset
import evaluate
from evaluate import evaluator
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, \
    DataCollatorWithPadding, pipeline

In [2]:
exist_path_train = 'data/EXIST2021_dataset/training/EXIST2021_training.tsv'
exist_path_test = 'data/EXIST2021_dataset/test/EXIST2021_test_labeled.tsv'

results_path = 'data/results/'

In [4]:
dataset = load_dataset("csv", data_files={"train": exist_path_train,
                        "test": exist_path_test}, sep="\t")

Using custom data configuration default-009c7012421518b0
Found cached dataset csv (/Users/jacoboj.pedrosa/.cache/huggingface/datasets/csv/default-009c7012421518b0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 2/2 [00:00<00:00, 135.29it/s]


In [5]:
dataset = dataset.map(lambda x: {"label":0} if x == 'non-sexist' else {"label":1})

100%|██████████| 4368/4368 [00:00<00:00, 7476.00ex/s]


In [6]:
es_dataset = dataset.filter(lambda example: example["language"] == 'es')

 80%|████████  | 4/5 [00:00<00:00, 37.32ba/s]


## Modelo dccuchile/bert-base-spanish-wwm-uncased

In [8]:
model_path = 'dccuchile/bert-base-spanish-wwm-uncased'

## Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

Downloading: 100%|██████████| 650/650 [00:00<00:00, 178kB/s]
Downloading: 100%|██████████| 248k/248k [00:00<00:00, 556kB/s] 
Downloading: 100%|██████████| 486k/486k [00:00<00:00, 952kB/s] 
Downloading: 100%|██████████| 134/134 [00:00<00:00, 56.5kB/s]


In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)

training_args = TrainingArguments(
    output_dir=results_path,
    per_device_train_batch_size=32,
    num_train_epochs=15,
    #fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

NameError: name 'results_path' is not defined